In [ ]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns

# Получение данных

In [ ]:
df = pd.read_csv('../input/heart-attack-analysis-prediction-dataset/heart.csv')

In [ ]:
df.head()

# Первичное исследование

In [ ]:
df.info()

In [ ]:
df[df.duplicated(keep=False)]

In [ ]:
df.drop_duplicates(inplace=True)  # удаляем дубль
df.shape

In [ ]:
df.dropna()
df.shape

In [ ]:
df.plot(subplots=True, figsize=(10, 30))

In [ ]:
import seaborn as sns
sns_plot = sns.pairplot(df)
sns_plot.savefig('./pairplot.png')  

In [ ]:
df.describe()

**Выводы:** 
* нашелся один дубликат
* пропусков нет, во всех 302 примерах данные заполнены.
* В данных почти нет выбросов кроме возможно высокого уровня холестерола, но его удалять не стал.

# Основное исследование

# Матрица корреляции

по матрице видим что с cp (тип боли в груди), thalachh (максимальный пульс?) прослеживается положительная зависимость
а по некоторым признаком довольно сильная обратная зависимость.

In [ ]:
import seaborn as sns


plt.subplots(figsize=(10,10))
sns.heatmap(df.corr(), fmt=".2f", annot=True, square=True)
plt.show()

наиболее ярко выраженные прямые зависимости

In [ ]:
columns_to_show = ['thalachh']
df.groupby(['output'])[columns_to_show].agg([np.mean]).plot(kind='bar')

In [ ]:
columns_to_show = ['cp', 'slp']
df.groupby(['output'])[columns_to_show].agg([np.mean]).plot(kind='bar')

наиболее ярко выраженные обратные зависимости

In [ ]:
columns_to_show = ['exng', 'oldpeak', 'caa', 'thall']
df.groupby(['output'])[columns_to_show].agg([np.mean]).plot(kind='bar')

судя по данным не видно сильно выраженной прямой зафисимости от возраста

In [ ]:
pd.crosstab(df['output'], df['age'])

In [ ]:
df['sex'].mean()

судя по данным женщины болеют чаще

In [ ]:
pd.crosstab(df['output'], df['sex'])

# Нормализация значений признаков

возраст и содержание холестирола отмаштабировать
эти параметры могут сильно повлиять при обучении методом KNN на расчет расстояний

как выяснилось в последствии когда запустил обучение с KNN это практически не повлияло на метрику, но алгоритм в целом дал очень плохие результаты


In [ ]:
X = np.array(df[df._get_numeric_data().drop(columns=['output']).columns])

In [ ]:
y = df['output'].values

In [ ]:
X_scaled = (X - X.T.mean())/X.T.std()

In [ ]:
X_scaled

# t-distributed Stohastic Neighbor Embedding

In [ ]:
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler



X_tdsne = df.drop(['output'], axis=1)

scaler = StandardScaler()
X_tdsne_scaled = scaler.fit_transform(X_tdsne)


tsne = TSNE(random_state=17)
tsne_representation = tsne.fit_transform(X_tdsne_scaled)

plt.scatter(tsne_representation[:, 0], tsne_representation[:, 1]);

раскрасим наше в множество и тех кто заболел оранжевым

In [ ]:
plt.scatter(tsne_representation[:, 0], tsne_representation[:, 1], 
            c=df['output'].map({0: 'blue', 1: 'orange'}));

очень круто!, видим что мы действительно можем выделить некоторую область в которой плотность заболевших намного больше,
также видим некоторые 'выбросы' оранжевых точек, которые сильно залезают в синию область

# Обучаем методом KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import dummy
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2)
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
preds = knn.predict(X_test)
accuracy_score(y_test, preds)

In [ ]:
dummy_clf = dummy.DummyClassifier("most_frequent").fit(X_train, y_train)
y_dummy = dummy_clf.predict(X_test)
accuracy_score(y_test, y_dummy)

получили очень плохие результаты, пробуем применить перебор гиперпараметров + кросвалидацию

In [ ]:
X_scaled

In [ ]:
from sklearn.model_selection import GridSearchCV

knn_grid = {'n_neighbors': np.array(np.linspace(5, 30, 25), dtype='int')}
gs = GridSearchCV(knn, knn_grid, cv=5, n_jobs=1) # используем кросвалидацию
gs.fit(X, y)

In [ ]:
def grid_plot(x, y, x_label, title, y_label='cross_val'):
    plt.figure(figsize=(12, 6))
    plt.grid(True)
    plt.plot(x, y, 'go-')
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)

In [ ]:
grid_plot(knn_grid['n_neighbors'], gs.cv_results_['mean_test_score'], 'n_neighbors', 'KNeighborsClassifier')

In [ ]:
print(gs.best_params_, gs.best_score_)

наилучший результат получили при 12 соседях, результаты по прежнему плохи

# Обучаем методом LogReg

* максимизируем recall
* используем кросвалидацию (маленький датасет)
* используем перебор гиперпараметра С (обратная сила регуляризации)

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(solver='liblinear', max_iter=1000)


logreg_grid = {'C': [1e-4, 1e-3, 1e-1, 1, 10]}
gs = GridSearchCV(
    logreg,
    logreg_grid,
    cv=5, # используем кросвалидацию
    n_jobs=-1,
    scoring=["accuracy", "precision", "recall", "f1"],
    refit='recall', # максимизируем recall
)
gs.fit(X, y)

print(gs.best_params_, gs.best_score_)

# Метрики

In [ ]:
fit_results = pd.DataFrame(gs.cv_results_)
fit_results[['params', 'mean_test_accuracy', 'mean_test_precision', 'mean_test_recall', 'mean_test_f1']]

Видим что при максимальном recall, метрика f1 также принимает максимальное значение, получили хороший результат в сравнении с KNN.

# Построим график ROC AUC

In [ ]:
from sklearn import metrics
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)
logreg = LogisticRegression(solver='liblinear', max_iter=100, C=0.1)
knn = KNeighborsClassifier(n_neighbors=12)
knn.fit(X_train, y_train)
logreg.fit(X_train, y_train)
y_logreg = logreg.predict_proba(X_test)
y_knn = knn.predict_proba(X_test)

In [ ]:
y_logreg_positive_prob = pd.DataFrame(y_logreg)[1].values
y_knn_positive_prob = pd.DataFrame(y_knn)[1].values

In [ ]:
plt.figure()


for index, res in enumerate([y_logreg_positive_prob, y_knn_positive_prob]):
    fpr, tpr, _ = metrics.roc_curve(y_test, res)
    plt.plot(fpr, tpr, label=f'ROC curve {index} (area = %0.2f)' % metrics.auc(fpr, tpr))

plt.grid(True)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('LogReg ROC AUC')
plt.legend(loc="lower right")
plt.show()


видим что в случае с knn получалось очень много результатов с одинаковыми вероятностями позитивных/негативных ответов из-за чего кривая для knn очень часто 'гадала' к чему отнести данный конкретный пример.